# 네이버 검색 결과 클롤링

In [ ]:
# 신촌역 맛집 / selenium을 활용하기

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import quote

In [3]:
# requests.get으로 데이터 가져오기
base_url = 'https://search.naver.com/search.naver?where=view&sm=tab_jum'
url = f'{base_url}&query={quote("신촌역 맛집")}'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [4]:
lis = soup.select('.api_subject_bx > ul > li')
len(lis)

64

In [5]:
lis[0]

<li class="bx" data-cr-area="rvw*x" data-cr-gdid="90000003_0000000000000033FCCC5D46" data-cr-rank="1"> <div class="view_wrap"> <div class="user_box"> <div class="api_save_group _keep_wrap"> <a aria-pressed="false" class="btn_save _keep_trigger" data-url="https://blog.naver.com/neomi4040/223284583750" href="#" onclick="tCR('a=rvw*x.kep&amp;r=1&amp;i=90000003_0000000000000033FCCC5D46&amp;u=javascript'); return false;" role="button"><i class="spnew ico_save">문서 저장하기</i></a> <div class="api_ly_save _keep_save_layer"> <a class="spnew_af item item_save _keep_save" data-cr-off="a=rvw*x.kepoff&amp;r=1&amp;i=90000003_0000000000000033FCCC5D46&amp;u=javascript" data-cr-on="a=rvw*x.kepon&amp;r=1&amp;i=90000003_0000000000000033FCCC5D46&amp;u=javascript" href="#" role="button">Keep에 저장</a> <a class="spnew_af item item_quick" href="https://keep.naver.com/" onclick="return goOtherCR(this,'a=rvw*x.kephome&amp;r=1&amp;i=90000003_0000000000000033FCCC5D46&amp;u='+urlencode(this.href));" target="_blank">Ke

# selenium 이용해서 데이터를 가져오기

In [6]:
from selenium import webdriver
driver = webdriver.Chrome()
driver.get(url)

In [7]:
# view 페이지를 아래로 내려서 많은 데이터 확보
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

body = driver.find_element(By.TAG_NAME, 'body')
for _ in range(15):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

In [8]:
soup1 = BeautifulSoup(driver.page_source)
lis = soup1.select('.view_wrap')
len(lis)

90

In [9]:
li = lis[3]

In [10]:
title = li.select_one('.title_link._cross_trigger').get_text().strip()
title

'신촌역 맛집 라구식당 파스타 라자냐 신촌 데이트'

In [11]:
contents = li.select_one('.dsc_link._cross_trigger').get_text().strip()
contents

'4. 총평 신촌역 맛집 라구식당 다녀왔어요!! 파스타와 라자냐가 유명한 레스토랑입니다. 신촌 데이트 하기에 최고의 분위기 가지고 있었어요. 여기에 와인, 맥주, 샹그리아 등 곁들일 수 있어서 연말 모임 장소 찾으시면 추천드려요!! #신촌역맛집 #라구식당 #신촌데이트'

In [12]:
date = li.select_one('.sub').get_text().strip()
date

'2023.11.01.'

In [13]:
from datetime import datetime, timedelta

now = datetime.now()
now

datetime.datetime(2023, 12, 13, 9, 18, 56, 63436)

In [14]:
# 어제
date = now - timedelta(days=1)
f'{date.year}.{date.month:02d}.{date.day:02d}'

'2023.12.12'

In [15]:
# 6일전
date = now - timedelta(days=6)
date.strftime('%Y.%m.%d') 

'2023.12.07'

In [17]:
# 40분전
date = now - timedelta(minutes=40)
date.strftime('%Y.%m.%d') 

'2023.12.13'

In [18]:
# 3주전 
date = now - timedelta(weeks=3)
date.strftime('%Y.%m.%d') 

'2023.11.22'

In [21]:
def convert_date(s):
    if len(s) == 11:    #'2023.12.13.'
        return s[:-1]
    
    now = datetime.now()
    if '어제' in s:
        date = now - timedelta(1)
    elif '일' in s:
        date = now - timedelta(int(s[0]))
    elif '시간' in s : 
        date = now - timedelta(hours=int(s[:-4]))
    elif '분' in s:
        date = now - timedelta(minutes=int(s[:-3]))
    elif '주 전' in s:
         date = now - timedelta(weeks=int(s[0]))
    
    return date.strftime('%Y.%m.%d') 


In [22]:
data = []
for li in lis:
    date = convert_date(li.select_one('.sub').get_text().strip())
    title = li.select_one('.title_link._cross_trigger').get_text().strip()
    contents = li.select_one('.dsc_link._cross_trigger').get_text().strip()
    data.append({'날짜' : date, '제목' : title, '내용': contents})

In [24]:
df = pd.DataFrame(data)
df.tail()

,날짜,제목,내용
85,2023.06.30,신촌역 맛집 최강 가성비의 고기집(통큰갈비 신촌본점),#신촌역맛집 #신촌역고기집 #통큰갈비신촌 신촌 중심가에서 살짝 떨어져 있는데요. 요...
86,2023.10.28,신촌역 맛집 소신이쏘 매운갈비찜,기본으로 김가루가 뿌려진 밥이 제공돼서 이거에 매운갈비찜 비벼먹으면 천국가는맛 ㅠ ...
87,2023.11.29,신촌점 피자몰 피자뷔페 신촌역맛집 추천 서울 아이와 가볼만한곳,오픈하자마자 갔더니 사람이 아직 들어오기 전이었는데 11시 조금 넘으니까 웨이팅이 ...
88,2023.12.06,[서울 서대문] 신촌역맛집 연대찜닭맛집 봉추찜닭 신촌점,#봉추찜닭 #신촌역찜닭 #연대찜닭 #연대맛집 #신촌맛집 #서울찜닭맛집 #신촌닭요리 ...
89,2023.11.15,신촌역 맛집 쭈꾸미블루스신촌점,#신촌맛집 #신촌 데이트 #신촌역맛집 #쭈꾸미블루스 #쭈꾸미블루스신촌본점 #쭈꾸미블...
